In [1]:
import utilities_geopandas as ugp

In [2]:
import geopandas as gpd
import os

In [3]:
# source: https://github.com/dongmeic/RTP/blob/main/analysis/process_parcel_data.ipynb
path = r'T:\Trans Projects\Model Development\UrbanSim_LandUse\Output\Simulation_47_Final_RTP'
shp = 'parcel_fabric.shp'
parcel_data = gpd.read_file(os.path.join(path, shp))

In [4]:
parcel_data.columns

Index(['btype', 'nrsqft', 'rsqft', 'du', 'yrbuilt', 'lpid', 'pundev',
       'dev_land', 'developed', 'obtype', 'orsqft', 'onrsqft', 'odu', 'zid',
       'rezoned', 'city', 'annexed', 'ozid', 'geometry'],
      dtype='object')

In [5]:
parcel_data['AreaPerJob'] = parcel_data['btype'].apply(lambda x: ugp.area_per_job(x)[0])

In [6]:
parcel_data['isNonRes'] = parcel_data['btype'].apply(lambda x: ugp.area_per_job(x)[1])

In [7]:
parcel_data['jobs'] = parcel_data.apply(lambda row: ugp.compute_jobs(row['isNonRes'],
                                                                     row['nrsqft'], row['AreaPerJob']), axis=1)
parcel_data['ojobs'] = parcel_data.apply(lambda row: ugp.compute_jobs(row['isNonRes'], 
                                                                      row['onrsqft'], row['AreaPerJob']), axis=1)

In [8]:
parcel_data['ndu'] = parcel_data['du'] - parcel_data['odu']
parcel_data['nnrsqft'] = parcel_data['nrsqft'] - parcel_data['onrsqft']

In [9]:
parcel_data['njobs'] = parcel_data.apply(lambda row: ugp.compute_jobs(row['isNonRes'], 
                                                                      row['nnrsqft'], row['AreaPerJob']), axis=1)

In [10]:
parcel_data['hh'] = parcel_data['du'].apply(lambda x: x*0.965)
parcel_data['ohh'] = parcel_data['odu'].apply(lambda x: x*0.965)
parcel_data['nhh'] = parcel_data['ndu'].apply(lambda x: x*0.965)

In [11]:
parcel_data = parcel_data.to_crs(epsg=3857)

In [12]:
parcel_data.to_file(os.path.join(path, 'output', 'parcel_data.shp'))

In [13]:
parcel_data.njobs.describe()

count    3924.000000
mean       23.833784
std       169.225725
min      -834.036279
25%         0.000000
50%         0.000000
75%         4.779313
max      6069.996603
Name: njobs, dtype: float64